# MR-ASH analysis on lung genes

Here we want to analyze with mr-ash some potentially interesting genes for a single tissue. We pick up e-genes for Lung from [previous results](https://www.gtexportal.org/home/multiTissuePage) -- I sort results by posterior probability in Lung and pick up the top 200 (have to manually save tables for batches of 100), saved to `data/lung_genes_v4.txt`.

## Get toy example


In [ ]:
!tail -n+2 data/lung_genes_v4.txt | cut -f 1 > /tmp/lung_genes.txt
 ./prep.sos make_toy -c conf/20170824.conf -b ~/Documents/GTEx/bin -J 8 -j 8

In [1]:
%run default -J8 -j8
import h5py
cwd = "~/Documents/GTEx/Lung"
genotype_file = '~/Documents/GTEx/Lung/Lung.genotype.h5'
expr_file = '~/Documents/GTEx/Lung/Lung.expr.h5'
covar_file = '~/Documents/GTEx/Lung/Lung.covariates.h5'
tissues = list(h5py.File("${expr_file!a}", libver='latest').keys())
f = h5py.File("${genotype_file!a}", libver='latest')
genes = []
for k in f:
    genes.extend(['{}/{}'.format(k, x) for x in f[k]])

[default]
sos_run('mr_ash', tissues = ['Lung'], genes = genes,
        genotype_file = genotype_file, 
        expr_file = expr_file, 
        covar_file = covar_file, 
        cwd = cwd)

[mr_ash]
parameter: cwd = "./"
parameter: genotype_file = ''
parameter: expr_file = ''
parameter: covar_file = ''
parameter: seed = 999
parameter: genes = []
parameter: tissues = []
depends: R_library('rhdf5'), R_library('tools')
input: for_each = ['genes', 'tissues']
output: "${cwd!a}/${_tissues}_${_genes!b}.rds"
task:
R:
load_data = function(genotype_file, expr_file, cov_file, geno_table, expr_table,cov_table) {
  geno = h5read(genotype_file, geno_table)
  gdata = geno$block0_values
  colnames(gdata) = geno$axis1
  rownames(gdata) = geno$axis0
  
  expr = h5read(expr_file, expr_table)
  edata = expr$block0_values
  # colnames(edata) = expr$axis1
  colnames(edata) = tools::file_path_sans_ext(expr$axis1)
  # rownames(edata) = expr$axis0
  rownames(edata) = apply(sapply(strsplit(expr$axis0,"-"), `[`, c(1,2)), 2, function(x) paste(x, collapse = '-'))
  
  index = which(duplicated(row.names(edata)))
  edata = edata[-index,]
  # edata = data.frame(edata)
  covariate <- h5read(cov_file, cov_table)
  cdata = covariate$block0_values
  colnames(cdata) = apply(sapply(strsplit(covariate$axis1,"-"), `[`, c(1,2)), 2, function(x) paste(x, collapse = '-'))
  # rownames(edata) = expr$axis0
  rownames(cdata) = covariate$axis0 
  cdata = t(cdata)[-index,]
  gdata = data.frame(gdata)
  # I want to use merge but later
  # index_overlap = which(row.names(gdata) %in% row.names(edata))
  edata = edata[, basename(geno_table)]
  edata = data.frame(edata)
  edata$ID = rownames(edata)
  gdata$ID = rownames(gdata)
  output = merge(x = edata, y = gdata, by = "ID", all.x = TRUE)
  # gdata = gdata[index_overlap,]
  return(list(X=as.matrix(output[,-c(1,2)]), y = as.vector(output$edata), Z = as.matrix(cdata)))
}

autoselect.mixsd = function(betahat,sebetahat,mult = sqrt(2)){
  sebetahat=sebetahat[sebetahat!=0] #To avoid exact measure causing (usually by mistake)
  sigmaamin = min(sebetahat)/10 #so that the minimum is small compared with measurement precision
  if(all(betahat^2<=sebetahat^2)){
    sigmaamax = 8*sigmaamin #to deal with the occassional odd case where this could happen; 8 is arbitrary
  }else{
    sigmaamax = 2*sqrt(max(betahat^2-sebetahat^2)) #this computes a rough largest value you'd want to use, based on idea that sigmaamax^2 + sebetahat^2 should be at least betahat^2   
  }
  if(mult==0){
    return(c(0,sigmaamax/2))
  }else{
    npoint = ceiling(log2(sigmaamax/sigmaamin)/log2(mult))
    return(mult^((-npoint):0) * sigmaamax)
  }
}

initial_step = function(X,y,Z = NULL){
  P = dim(X)[2]
  output = matrix(0,nrow = P,ncol = 2)
  for(i in 1:P){
    if(is.null(Z)){
      g = summary(lm(y~X[,i]))
    } else{
      g = summary(lm(y~X[,i]+Z))
    }
    
    output[i,] = g$coefficients[2,1:2]
  }
  return(list(betahat = output[,1],sebetahat = output[,2]))
}
                                                                  
analyze = function(genename = '/chr4/ENSG00000145214', tissue = '/Lung', out = 'test.rds'){
  library(rhdf5)
  genotype_file = ${genotype_file!ar}
  expr_file = ${expr_file!ar}
  geno_table = genename
  expr_table = tissue 
  gene = basename(geno_table)
  cov_file = ${covar_file!ar}
  cov_table = expr_table
  dat = load_data(genotype_file = genotype_file,
                  expr_file = expr_file,
                  cov_file = cov_file,
                  geno_table = geno_table,
                  expr_table = expr_table,
                  cov_table = cov_table)
  X = as.matrix(dat$X)
  X = X[,which(colSums(X)!=0)]
  if ((nrow(X) == 0) || (ncol(X) == 0)) {
  	saveRDS(list(), out)
  } else {
  storage.mode(X) <- "double"
  y = as.vector(dat$y)
  Z = as.matrix(dat$Z)
  initial = initial_step(X,y,Z)
  mixsd = autoselect.mixsd(initial$betahat,initial$sebetahat)
  logdata = capture.output({res = varbvs::varbvsmix(X, Z, y, sa = c(0,mixsd^2)) })
  betahat = rowSums(res$alpha * res$mu)
  names(betahat) = colnames(X)
  mrash_out = list(betahat = betahat, lfsr = res$lfsr)
  ash_out = ashr::ash(initial$betahat,initial$sebetahat,mixcompdist = "normal")
  saveRDS(list(ash = ash_out, uni = initial, mr_ash = mrash_out, logdata = logdata), out)
}
}
set.seed(999)
analyze(genename = "/${_genes}", tissue = "/${_tissues}", out = ${_output!r})

"<i id=""status_localhost_80341bb31855f4538f8d926145818e6c"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_80341bb31855f4538f8d926145818e6c').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_80341bb31855f4538f8d926145818e6c').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('80341bb31855f4538f8d926145818e6c', 'localhost')"" >",80341bb31855f4538f8d926145818e6c,,0 sec


"<i id=""status_localhost_0c1b5684a800c4501bdd1d20ce8c3798"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_0c1b5684a800c4501bdd1d20ce8c3798').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_0c1b5684a800c4501bdd1d20ce8c3798').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('0c1b5684a800c4501bdd1d20ce8c3798', 'localhost')"" >",0c1b5684a800c4501bdd1d20ce8c3798,,0 sec


"<i id=""status_localhost_b04b7140556bf187c0c275efde8504bc"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_b04b7140556bf187c0c275efde8504bc').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_b04b7140556bf187c0c275efde8504bc').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('b04b7140556bf187c0c275efde8504bc', 'localhost')"" >",b04b7140556bf187c0c275efde8504bc,,0 sec


"<i id=""status_localhost_0044bdff4eabc8c3701bc54dcfea4d34"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_0044bdff4eabc8c3701bc54dcfea4d34').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_0044bdff4eabc8c3701bc54dcfea4d34').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('0044bdff4eabc8c3701bc54dcfea4d34', 'localhost')"" >",0044bdff4eabc8c3701bc54dcfea4d34,,0 sec


"<i id=""status_localhost_ff79a14268a6ce829599e50f2e35b09b"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_ff79a14268a6ce829599e50f2e35b09b').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_ff79a14268a6ce829599e50f2e35b09b').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('ff79a14268a6ce829599e50f2e35b09b', 'localhost')"" >",ff79a14268a6ce829599e50f2e35b09b,,0 sec


...

"<i id=""status_localhost_4506b359b850c721ede9ff64bbfea74a"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_4506b359b850c721ede9ff64bbfea74a').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_4506b359b850c721ede9ff64bbfea74a').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('4506b359b850c721ede9ff64bbfea74a', 'localhost')"" >",4506b359b850c721ede9ff64bbfea74a,,0 sec


"<i id=""status_localhost_56af1627a19c7d641e38c9f66237b195"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_56af1627a19c7d641e38c9f66237b195').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_56af1627a19c7d641e38c9f66237b195').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('56af1627a19c7d641e38c9f66237b195', 'localhost')"" >",56af1627a19c7d641e38c9f66237b195,,0 sec


"<i id=""status_localhost_9a8e5818732daa7be73568b454cab29d"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_9a8e5818732daa7be73568b454cab29d').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_9a8e5818732daa7be73568b454cab29d').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('9a8e5818732daa7be73568b454cab29d', 'localhost')"" >",9a8e5818732daa7be73568b454cab29d,,0 sec


"<i id=""status_localhost_aad6997f879c6b72b1a6465d5d572ada"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_aad6997f879c6b72b1a6465d5d572ada').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_aad6997f879c6b72b1a6465d5d572ada').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('aad6997f879c6b72b1a6465d5d572ada', 'localhost')"" >",aad6997f879c6b72b1a6465d5d572ada,,0 sec


"<i id=""status_localhost_cb0ed7ea0c70f83216ac161b16057f8f"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_cb0ed7ea0c70f83216ac161b16057f8f').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_cb0ed7ea0c70f83216ac161b16057f8f').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('cb0ed7ea0c70f83216ac161b16057f8f', 'localhost')"" >",cb0ed7ea0c70f83216ac161b16057f8f,,0 sec
